# Proyecto: Agrupación y filtrado de información en RDDs

En la presente práctica veremos cómo:

* crear RDDs a partir de archivos .csv 

* seleccionar una columna específica de un RDD

Algunas funciones que veremos son:

`.distinct()` Extrae las clases de datos de una columna dada

`.count()` Realiza un conteo de registros de un RDD

`.map( lambda x: (x[i],x[j]) ).groupByKey().mapValues(list)` Agrupa la información de un RDD en forma llave-valor, y regresa una lista donde las llaves se toman de 'x[i]' y los valores se roamn de 'x[j]'

`.map( lambda x: (x[i],x[j]) ).groupByKey().mapValues(len)` Agrupa la información de un RDD en forma llave-valor, y regresa una lista que muestra el número de valores 'x[j]' correspondiente a la llave 'x[i]'

`.filter(lambda l : 'some_word' in l).collect()` Filtra los registros que contienen el valor 'some_word'

`.countApprox(m)` Realiza un conteo de registros en un RDD, donde la operación de conteo se realiza tiene un límite de tiempo dado por 'm' miliseconds


### OBSERVACIÓN: La función `.collect()` recolecta toda la información distribuida en el cluster y la trae al nodo que ejecuta el `.collect()` Esto puede ser peligroso ya que recolectar una gran cantidad de información en un sólo nodo puede sobrepase la memoria computacional del nodo y hacer que el sistema colapse.

In [1]:
from pyspark import SparkContext

In [2]:
# Creamos el incializador viviendo en nuestra computadora 'local'
sc = SparkContext(master='local', appName = 'transformacionesYAcciones')

## Creación de un RDD cargando un archivo .csv

In [3]:
# Visualizamos el contenido de la carpeta 'Data' 
!ls ./Data/

deporte.csv	 deportistaError.csv  modelo_relacional.jpg  salidatexto
deportista2.csv  evento.csv	      paises.csv
deportista.csv	 juegos.csv	      resultados.csv


In [4]:
# Ruta en donde se encuentran los datos con los que trabajaremos:
path = './Data/'

In [5]:
# Creamos un RDD con los datos del archivo 'paises.csv':

equiposOlimpicosRDD = sc.textFile(path+'paises.csv').map(lambda line : line.split(','))

# La primer parte de código '# equiposOlimpicosRDD = sc.textFile(path+'paises.csv')'
# carga los datos sin darle un formato (carga los datos como una columna)
# El resto de código '.map(lambda line : line.split(','))' le asigna un formato a los datos,
# y como se trata de un archivo .csv  sabemos que los elementos de un renglón están separados por una coma (,)

In [6]:
# Visualizamos los primeros N registros de RDD
# Hay que notar que el encabezado cuenta como primer registro
N=10
equiposOlimpicosRDD.take(N)

[['id', 'equipo', 'sigla'],
 ['1', '30. Februar', 'AUT'],
 ['2', 'A North American Team', 'MEX'],
 ['3', 'Acipactli', 'MEX'],
 ['4', 'Acturus', 'ARG'],
 ['5', 'Afghanistan', 'AFG'],
 ['6', 'Akatonbo', 'IRL'],
 ['7', 'Alain IV', 'SUI'],
 ['8', 'Albania', 'ALB'],
 ['9', 'Alcaid', 'POR']]

In [13]:
# x[0]   <-- selecciona la columna 'id'
# x[1]   <-- selecciona la columna 'equipo'
# x[2]   <-- selecciona la columna 'sigla'

# Extraemos los primeros N registros de una columna en particular:
equiposOlimpicosRDD.map(lambda x: (x[2]) ).take(N)


['sigla', 'AUT', 'MEX', 'MEX', 'ARG', 'AFG', 'IRL', 'SUI', 'ALB', 'POR']

In [15]:
# Extraemos las primeras N clases de valores de una columna en particular:
equiposOlimpicosRDD.map(lambda x: (x[2]) ).distinct().take(N)


['sigla', 'AUT', 'MEX', 'ARG', 'AFG', 'IRL', 'SUI', 'ALB', 'POR', 'FRA']

In [25]:
# Realizamos el conteo de las clases de valores de una columna en particular:
equiposOlimpicosRDD.map(lambda x: (x[2]) ).distinct().count()


231

## Algunas operaciones con agrupación de datos en RDDs

In [26]:
N = 5 # <-- Número de registros a mostrar

In [30]:
equiposOlimpicosRDD.map( lambda x: (x[2],x[1]) ).groupByKey().mapValues(list).take(N)

# .map( lambda x: (x[i],x[j]) ).groupByKey().mapValues(list).take(N) 
# Agrupa la información de un RDD en forma llave-valor, y regresa una lista 
# donde las llaves se toman de 'x[i]' y los valores se roamn de 'x[j]'

[('sigla', ['equipo']),
 ('AUT',
  ['30. Februar',
   'Austria',
   'Austria-1',
   'Austria-2',
   'Breslau',
   'Brigantia',
   'Donar III',
   'Evita VI',
   'May-Be 1960',
   '"R.-V. Germania; Leitmeritz"',
   'Surprise']),
 ('MEX',
  ['A North American Team',
   'Acipactli',
   'Chamukina',
   'Mexico',
   'Mexico-1',
   'Mexico-2',
   'Nausikaa 4',
   'Tlaloc',
   'Xolotl']),
 ('ARG',
  ['Acturus',
   'Antares',
   'Arcturus',
   'Ardilla',
   'Argentina',
   'Argentina-1',
   'Argentina-2',
   'Blue Red',
   'Covunco III',
   'Cupidon III',
   'Djinn',
   'Gullvinge',
   'Matrero II',
   'Mizar',
   'Pampero',
   'Rampage',
   'Tango',
   'Wiking']),
 ('AFG', ['Afghanistan'])]

In [31]:
# Para la agrupación por llaves, ejecutamos '.groupByKey()'
# aplicado a una función `lambda x: (llave_x, valor_x)`


equiposOlimpicosRDD.map( lambda x: (x[2],x[1]) ).groupByKey().mapValues(len).take(N)

# .map( lambda x: (x[j],x[k]) ).groupByKey().mapValues(len).take(N) 
# Agrupa los datos con la jerarquía indicada por las columnas (x[j],x[k]) y 
# mapValues(list).take(N) muestra la llave x[j] y su correspondiente lista de valores en x[k]

[('sigla', 1), ('AUT', 11), ('MEX', 9), ('ARG', 18), ('AFG', 1)]

## Filtrado de información en un RDD

In [33]:
# Extraer información unicamente de los equipos de Argentina 'ARG'

aquiposArgentinos = equiposOlimpicosRDD.filter(lambda l : 'ARG' in l).collect()
aquiposArgentinos

[['4', 'Acturus', 'ARG'],
 ['37', 'Antares', 'ARG'],
 ['42', 'Arcturus', 'ARG'],
 ['43', 'Ardilla', 'ARG'],
 ['45', 'Argentina', 'ARG'],
 ['46', 'Argentina-1', 'ARG'],
 ['47', 'Argentina-2', 'ARG'],
 ['119', 'Blue Red', 'ARG'],
 ['238', 'Covunco III', 'ARG'],
 ['252', 'Cupidon III', 'ARG'],
 ['288', 'Djinn', 'ARG'],
 ['436', 'Gullvinge', 'ARG'],
 ['644', 'Matrero II', 'ARG'],
 ['672', 'Mizar', 'ARG'],
 ['774', 'Pampero', 'ARG'],
 ['843', 'Rampage', 'ARG'],
 ['1031', 'Tango', 'ARG'],
 ['1162', 'Wiking', 'ARG']]

# OBSERVACIÓN: La función `.collect()` recolecta toda la información distribuida en el cluster y la trae al nodo que ejecuta el `.collect()` Esto puede ser peligroso ya que recolectar una gran cantidad de información en un sólo nodo puede sobrepase la memoria computacional del nodo y hacer que el sistema colapse.

In [34]:
# debido a que el conteo '.count()' de registros en RDDs grandes puede ser tardado, 
# podemos definir un limite de tiempo para realizar el conteo.
# Una vez que se alcanza el limite de tiempo se aborta la opercación de conteo.

limit_time = 20 #<-- limite de tiempo en milisegundos

equiposOlimpicosRDD.countApprox(limit_time)

1185

In [35]:
equiposOlimpicosRDD.count()

1185

In [36]:
# Cerramos la sesión para liberar memoria:
sc.stop()